## Question 1

In [1]:
# 1)
import findspark
findspark.init()

import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
from pyspark.sql.types import IntegerType
spark = SparkSession \
    .builder \
    .appName("ReadNumbers") \
    .getOrCreate()

lines = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .load()

# Split the lines into words
nums = lines.select(
   explode(
       split(lines.value, ",")
   ).alias("number")
)
numbers = nums.withColumn("numbers", nums["number"].cast(IntegerType())).drop('number')
numbers.printSchema()

root
 |-- numbers: integer (nullable = true)



In [2]:
# 2)
from pyspark.sql import SQLContext
sqlContext = SQLContext(spark)
query1 = numbers \
    .writeStream \
    .queryName("Query1") \
    .outputMode("append") \
    .format("memory") \
    .start()

In [3]:
query1.stop()

In [4]:
# 3)
part_3 = sqlContext.sql("SELECT * FROM Query1 ORDER BY numbers ASC")
part_3.show()

+-------+
|numbers|
+-------+
|    -20|
|     10|
|     40|
+-------+



In [8]:
# 4)
part_4 = sqlContext.sql("SELECT MIN(numbers) AS minimum, MAX(numbers) AS maximum, AVG(numbers) AS average FROM Query1")
part_4.show()

+-------+-------+-------+
|minimum|maximum|average|
+-------+-------+-------+
|    -20|     40|   10.0|
+-------+-------+-------+

